In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
root_dir = '/home/biyanbahtiar/portfolio/ai-dashboard-app/project/'

In [3]:
metrics_raw_data = pd.read_csv(os.path.join(root_dir, 'data/processed/metrics_raw_data.csv'))
metrics_raw_data.head()

,Unnamed: 0,Date,Year,Q,Month,Week,Year_Quarter,Product_ID,Warehouse_Loc_ID,Daily_Sales,Forecasted_Demand,Stock_Level,Inventory_Holding_Cost,Lost_Sales
0,0,2023-12-31,2023,4,12,53,2023-Q4,PROD1089,Surabaya-WH10,14,274,147,40567.57,3
1,1,2023-12-31,2023,4,12,53,2023-Q4,PROD1110,Surabaya-WH10,11,393,445,41648.06,10
2,2,2023-12-31,2023,4,12,53,2023-Q4,PROD1111,Surabaya-WH10,3,414,376,29393.83,8
3,3,2023-12-31,2023,4,12,53,2023-Q4,PROD1118,Surabaya-WH10,4,963,189,44976.58,6
4,4,2023-12-31,2023,4,12,53,2023-Q4,PROD1023,Bali-WH11,7,584,257,20015.87,5


In [6]:
metrics_raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1958040 entries, 0 to 1958039
Data columns (total 14 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Unnamed: 0              int64  
 1   Date                    object 
 2   Year                    int64  
 3   Q                       int64  
 4   Month                   int64  
 5   Week                    int64  
 6   Year_Quarter            object 
 7   Product_ID              object 
 8   Warehouse_Loc_ID        object 
 9   Daily_Sales             int64  
 10  Forecasted_Demand       int64  
 11  Stock_Level             int64  
 12  Inventory_Holding_Cost  float64
 13  Lost_Sales              int64  
dtypes: float64(1), int64(9), object(4)
memory usage: 209.1+ MB


In [23]:
sorted(metrics_raw_data['Year_Quarter'].unique())

['2023-Q1',
 '2023-Q2',
 '2023-Q3',
 '2023-Q4',
 '2024-Q1',
 '2024-Q2',
 '2024-Q3',
 '2024-Q4']

### Test

In [7]:
metrics_raw_data['Date'] = pd.to_datetime(metrics_raw_data['Date'])
itr = metrics_raw_data.groupby('Year_Quarter', as_index=False).agg({'Date':['min','max'], 'Daily_Sales':'sum'})
itr.columns = ['_'.join(col).strip() if col[1] else col[0] for col in itr.columns.values]

earliest_stock_sum = [(metrics_raw_data[metrics_raw_data['Date']==i]['Stock_Level'].sum()) for i in itr['Date_min']]
latest_stock_sum = [(metrics_raw_data[metrics_raw_data['Date']==i]['Stock_Level'].sum()) for i in itr['Date_max']]

itr['Earliest_Stock_Sum'] = earliest_stock_sum
itr['Latest_Stock_Sum'] = latest_stock_sum

itr['ITR'] = (itr['Daily_Sales_sum']/((itr['Latest_Stock_Sum']+itr['Earliest_Stock_Sum'])/2)).round(2)
itr.drop(columns=['Date_min','Date_max', 'Daily_Sales_sum', 'Earliest_Stock_Sum', 'Latest_Stock_Sum'], inplace=True)

In [ ]:
itr_yr_q = sorted(itr['Year_Quarter'].unique())
latest_itr = itr[itr['Year_Quarter'] == itr['Year_Quarter'].max()]['ITR'].values[0]
latestmin1_itr = itr[itr['Year_Quarter'] == itr_yr_q[-2]]['ITR'].values[0]

### ITR

In [6]:
# Calculate itr
itr = metrics_raw_data.groupby(['Year_Quarter', 'Product_ID'], as_index=False).agg({'Date':['min','max'], 'Daily_Sales':'sum'})
itr.columns = ['_'.join(col).strip() if col[1] else col[0] for col in itr.columns.values]
itr = itr[itr['Year_Quarter']=='2024-Q4']

unique_product = itr['Product_ID'].unique()

earliest_stock_sum_dict = {}
latest_stock_sum_dict = {}

for prod in unique_product:
    metrics_raw_datemin = metrics_raw_data[(metrics_raw_data['Product_ID']==prod) & (metrics_raw_data['Date'].isin(itr['Date_min']))]
    earliest_stock_sum = metrics_raw_datemin['Stock_Level'].sum()
    earliest_stock_sum_dict[prod] = earliest_stock_sum

    metrics_raw_datemax = metrics_raw_data[(metrics_raw_data['Product_ID']==prod) & (metrics_raw_data['Date'].isin(itr['Date_max']))]
    latest_stock_sum = metrics_raw_datemax['Stock_Level'].sum()
    latest_stock_sum_dict[prod] = latest_stock_sum

print(earliest_stock_sum_dict)

{'PROD1000': 5958, 'PROD1001': 4784, 'PROD1002': 5624, 'PROD1003': 5227, 'PROD1004': 4752, 'PROD1005': 5436, 'PROD1006': 6571, 'PROD1007': 5385, 'PROD1008': 4214, 'PROD1009': 4756, 'PROD1010': 4037, 'PROD1011': 5405, 'PROD1012': 4325, 'PROD1013': 5402, 'PROD1014': 5932, 'PROD1015': 5971, 'PROD1016': 4773, 'PROD1017': 5425, 'PROD1018': 5749, 'PROD1019': 4940, 'PROD1020': 5554, 'PROD1021': 4479, 'PROD1022': 4871, 'PROD1023': 4536, 'PROD1024': 4623, 'PROD1025': 6024, 'PROD1026': 4966, 'PROD1027': 5519, 'PROD1028': 5230, 'PROD1029': 4004, 'PROD1030': 5187, 'PROD1031': 6340, 'PROD1032': 4362, 'PROD1033': 5569, 'PROD1034': 4253, 'PROD1035': 4823, 'PROD1036': 3761, 'PROD1037': 4704, 'PROD1038': 5813, 'PROD1039': 4578, 'PROD1040': 5917, 'PROD1041': 5268, 'PROD1042': 5352, 'PROD1043': 4853, 'PROD1044': 4835, 'PROD1045': 5278, 'PROD1046': 5294, 'PROD1047': 4481, 'PROD1048': 5632, 'PROD1049': 5710, 'PROD1050': 5707, 'PROD1051': 5499, 'PROD1052': 4581, 'PROD1053': 4983, 'PROD1054': 4828, 'PROD1055

In [7]:
itr['Earliest_Stock_Sum'] = itr['Product_ID'].map(earliest_stock_sum_dict)
itr['Latest_Stock_Sum'] = itr['Product_ID'].map(latest_stock_sum_dict)

itr['ITR'] = (itr['Daily_Sales_sum']/((itr['Latest_Stock_Sum']+itr['Earliest_Stock_Sum'])/2)).round(2)
itr.drop(columns=['Date_min','Date_max', 'Daily_Sales_sum', 'Earliest_Stock_Sum', 'Latest_Stock_Sum'], inplace=True)

display(itr)

,Year_Quarter,Product_ID,ITR
1004,2024-Q4,PROD1000,4.79
1005,2024-Q4,PROD1001,5.49
1006,2024-Q4,PROD1002,4.81
1007,2024-Q4,PROD1003,5.39
1008,2024-Q4,PROD1004,6.00
...,...,...,...
1149,2024-Q4,PROD1145,4.88
1150,2024-Q4,PROD1146,5.87
1151,2024-Q4,PROD1147,6.31
1152,2024-Q4,PROD1148,5.11


### Stockout Ratio

In [8]:
# Calculate Stockout Ratio per Year-Quarter
stockout_ratio = metrics_raw_data.groupby(['Year_Quarter', 'Product_ID'], as_index=False).agg({'Daily_Sales':'sum', 'Lost_Sales':'sum'})
stockout_ratio = stockout_ratio[stockout_ratio['Year_Quarter']=='2024-Q4']    
stockout_ratio['stockout_ratio'] = round((stockout_ratio['Lost_Sales'] / (stockout_ratio['Lost_Sales'] + stockout_ratio['Daily_Sales'])) * 100, 2)
stockout_ratio.drop(columns=['Daily_Sales', 'Lost_Sales'], inplace=True)

display(stockout_ratio)

,Year_Quarter,Product_ID,stockout_ratio
1004,2024-Q4,PROD1000,16.46
1005,2024-Q4,PROD1001,16.51
1006,2024-Q4,PROD1002,16.81
1007,2024-Q4,PROD1003,17.06
1008,2024-Q4,PROD1004,16.37
...,...,...,...
1149,2024-Q4,PROD1145,17.23
1150,2024-Q4,PROD1146,16.42
1151,2024-Q4,PROD1147,16.27
1152,2024-Q4,PROD1148,16.25


### Overstock Cost

In [10]:
metrics_filtered = metrics_raw_data[metrics_raw_data['Year_Quarter'] == '2024-Q4']

overstock_cost = (
    metrics_filtered.groupby(['Year_Quarter', 'Product_ID', 'Date'], as_index=False)
    .agg({
        'Daily_Sales': 'sum',
        'Stock_Level': 'sum',
        'Inventory_Holding_Cost': 'mean'
    })
)

overstock_cost = (
    overstock_cost.sort_values('Date')
    .groupby(['Year_Quarter', 'Product_ID'], as_index=False)
    .agg({
        'Date': 'max',
        'Daily_Sales': 'mean',
        'Stock_Level': 'last',
        'Inventory_Holding_Cost': 'last'
    })
)
overstock_cost['Inventory_Holding_Cost'] = overstock_cost['Inventory_Holding_Cost'].round(2)
overstock_cost['Daily_Sales'] = overstock_cost['Daily_Sales'].round(2)
overstock_cost['overstock_cost'] = (overstock_cost['Stock_Level'] - overstock_cost['Daily_Sales']) * overstock_cost['Inventory_Holding_Cost']
overstock_cost['overstock_cost'] = overstock_cost['overstock_cost'].round(2)
overstock_cost.drop(columns=['Daily_Sales', 'Stock_Level', 'Date', 'Inventory_Holding_Cost'], inplace=True)

display(overstock_cost)


,Year_Quarter,Product_ID,overstock_cost
0,2024-Q4,PROD1000,1.157823e+08
1,2024-Q4,PROD1001,1.129774e+08
2,2024-Q4,PROD1002,1.284583e+08
3,2024-Q4,PROD1003,8.744550e+07
4,2024-Q4,PROD1004,7.949261e+07
...,...,...,...
145,2024-Q4,PROD1145,1.064182e+08
146,2024-Q4,PROD1146,7.232052e+07
147,2024-Q4,PROD1147,9.466142e+07
148,2024-Q4,PROD1148,1.096538e+08


### Concat all

In [11]:
concat_df = itr.merge(stockout_ratio, on=['Product_ID', 'Year_Quarter']).merge(overstock_cost, on=['Product_ID', 'Year_Quarter'])

display(concat_df)

,Year_Quarter,Product_ID,ITR,stockout_ratio,overstock_cost
0,2024-Q4,PROD1000,4.79,16.46,1.157823e+08
1,2024-Q4,PROD1001,5.49,16.51,1.129774e+08
2,2024-Q4,PROD1002,4.81,16.81,1.284583e+08
3,2024-Q4,PROD1003,5.39,17.06,8.744550e+07
4,2024-Q4,PROD1004,6.00,16.37,7.949261e+07
...,...,...,...,...,...
145,2024-Q4,PROD1145,4.88,17.23,1.064182e+08
146,2024-Q4,PROD1146,5.87,16.42,7.232052e+07
147,2024-Q4,PROD1147,6.31,16.27,9.466142e+07
148,2024-Q4,PROD1148,5.11,16.25,1.096538e+08


In [12]:
concat_df['ITR_norm'] = (
    (concat_df['ITR'] - concat_df['ITR'].min()) /
    (concat_df['ITR'].max() - concat_df['ITR'].min())
)

concat_df['stockout_ratio_norm'] = (
    (concat_df['stockout_ratio'] - concat_df['stockout_ratio'].min()) /
    (concat_df['stockout_ratio'].max() - concat_df['stockout_ratio'].min())
)

concat_df['overstock_cost_norm'] = (
    (concat_df['overstock_cost'] - concat_df['overstock_cost'].min()) /
    (concat_df['overstock_cost'].max() - concat_df['overstock_cost'].min())
)

display(concat_df)

,Year_Quarter,Product_ID,ITR,stockout_ratio,overstock_cost,ITR_norm,stockout_ratio_norm,overstock_cost_norm
0,2024-Q4,PROD1000,4.79,16.46,1.157823e+08,0.190000,0.371795,0.455826
1,2024-Q4,PROD1001,5.49,16.51,1.129774e+08,0.423333,0.393162,0.431355
2,2024-Q4,PROD1002,4.81,16.81,1.284583e+08,0.196667,0.521368,0.566413
3,2024-Q4,PROD1003,5.39,17.06,8.744550e+07,0.390000,0.628205,0.208611
4,2024-Q4,PROD1004,6.00,16.37,7.949261e+07,0.593333,0.333333,0.139229
...,...,...,...,...,...,...,...,...
145,2024-Q4,PROD1145,4.88,17.23,1.064182e+08,0.220000,0.700855,0.374131
146,2024-Q4,PROD1146,5.87,16.42,7.232052e+07,0.550000,0.354701,0.076659
147,2024-Q4,PROD1147,6.31,16.27,9.466142e+07,0.696667,0.290598,0.271564
148,2024-Q4,PROD1148,5.11,16.25,1.096538e+08,0.296667,0.282051,0.402360


### Save

In [ ]:
concat_df.to_csv(os.path.join(root_dir, 'data/processed/metrics_barpolar.csv'))

In [13]:
concat_df.nlargest(10, 'ITR_norm')

,Year_Quarter,Product_ID,ITR,stockout_ratio,overstock_cost,ITR_norm,stockout_ratio_norm,overstock_cost_norm
131,2024-Q4,PROD1131,7.22,17.10,89430599.44,1.000000,0.645299,0.225930
8,2024-Q4,PROD1008,6.59,16.76,67256808.07,0.790000,0.500000,0.032483
69,2024-Q4,PROD1069,6.52,16.58,92967948.18,0.766667,0.423077,0.256790
132,2024-Q4,PROD1132,6.49,16.41,83068868.33,0.756667,0.350427,0.170429
29,2024-Q4,PROD1029,6.43,16.72,94206352.42,0.736667,0.482906,0.267594
125,2024-Q4,PROD1125,6.36,16.78,79005352.85,0.713333,0.508547,0.134978
89,2024-Q4,PROD1089,6.34,17.20,64521326.28,0.706667,0.688034,0.008618
147,2024-Q4,PROD1147,6.31,16.27,94661419.76,0.696667,0.290598,0.271564
57,2024-Q4,PROD1057,6.30,16.62,93454031.81,0.693333,0.440171,0.261031
97,2024-Q4,PROD1097,6.19,16.53,79840953.37,0.656667,0.401709,0.142268


### Refactor

In [10]:
metrics_raw_data.head(1)

,Unnamed: 0,Date,Year,Q,Month,Week,Year_Quarter,Product_ID,Warehouse_Loc_ID,Daily_Sales,Forecasted_Demand,Stock_Level,Inventory_Holding_Cost,Lost_Sales
0,0,2023-12-31,2023,4,12,53,2023-Q4,PROD1007,Surabaya-WH1,8,516,425,43490.71,9


In [ ]:
def preprocess_barpolar(metrics_raw_data):
    def calculate_inventory_turnover_ratio(metrics_raw_data: pd.DataFrame, target_quarter: str):
        # Group by quarter and product to get sales and date ranges
        inventory_metrics = metrics_raw_data.groupby(
            ['Year_Quarter', 'Product_ID'], 
            as_index=False
        ).agg({
            'Date': ['min', 'max'],
            'Daily_Sales': 'sum'
        })
        
        # Clean up column names
        inventory_metrics.columns = ['_'.join(col).strip() if col[1] else col[0] 
                                for col in inventory_metrics.columns.values]
        
        # Filter for target quarter
        inventory_metrics = inventory_metrics[inventory_metrics['Year_Quarter'] == target_quarter]
        
        # Calculate stock levels for start and end dates
        stock_levels = {}
        for date_type in ['min', 'max']:
            stock_levels[date_type] = {
                prod: metrics_raw_data[
                    (metrics_raw_data['Product_ID'] == prod) & 
                    (metrics_raw_data['Date'].isin(inventory_metrics[f'Date_{date_type}']))
                ]['Stock_Level'].sum()
                for prod in inventory_metrics['Product_ID'].unique()
            }
        
        # Add stock levels to dataframe
        inventory_metrics['Earliest_Stock_Sum'] = inventory_metrics['Product_ID'].map(stock_levels['min'])
        inventory_metrics['Latest_Stock_Sum'] = inventory_metrics['Product_ID'].map(stock_levels['max'])
        
        # Calculate ITR
        inventory_metrics['ITR'] = (
            inventory_metrics['Daily_Sales_sum'] / 
            ((inventory_metrics['Latest_Stock_Sum'] + inventory_metrics['Earliest_Stock_Sum']) / 2)
        ).round(2)
        
        return inventory_metrics[['Year_Quarter', 'Product_ID', 'ITR']]

    def calculate_stockout_ratio(metrics_raw_data: pd.DataFrame, target_quarter: str):
        stockout_metrics = metrics_raw_data.groupby(
            ['Year_Quarter', 'Product_ID'], 
            as_index=False
        ).agg({
            'Daily_Sales': 'sum',
            'Lost_Sales': 'sum'
        })
        
        stockout_metrics = stockout_metrics[stockout_metrics['Year_Quarter'] == target_quarter]
        
        stockout_metrics['stockout_ratio'] = (
            (stockout_metrics['Lost_Sales'] / 
            (stockout_metrics['Lost_Sales'] + stockout_metrics['Daily_Sales'])) * 100
        ).round(2)
        
        return stockout_metrics[['Year_Quarter', 'Product_ID', 'stockout_ratio']]

    def calculate_overstock_cost(metrics_raw_data: pd.DataFrame, target_quarter: str):
        quarterly_data = metrics_raw_data[metrics_raw_data['Year_Quarter'] == target_quarter]
        
        # Calculate daily metrics
        daily_metrics = quarterly_data.groupby(
            ['Year_Quarter', 'Product_ID', 'Date'], 
            as_index=False
        ).agg({
            'Daily_Sales': 'sum',
            'Stock_Level': 'sum',
            'Inventory_Holding_Cost': 'mean'
        })
        
        # Calculate final metrics
        overstock_metrics = (
            daily_metrics.sort_values('Date')
            .groupby(['Year_Quarter', 'Product_ID'], as_index=False)
            .agg({
                'Daily_Sales': 'mean',
                'Stock_Level': 'last',
                'Inventory_Holding_Cost': 'last'
            })
        )
        
        # Calculate overstock cost
        overstock_metrics['overstock_cost'] = (
            (overstock_metrics['Stock_Level'] - overstock_metrics['Daily_Sales']) * 
            overstock_metrics['Inventory_Holding_Cost']
        ).round(2)
        
        return overstock_metrics[['Year_Quarter', 'Product_ID', 'overstock_cost']]
    
    target_quarter = '2024-Q4'
    itr_df = calculate_inventory_turnover_ratio(metrics_raw_data, target_quarter)
    stockout_df = calculate_stockout_ratio(metrics_raw_data, target_quarter)
    overstock_df = calculate_overstock_cost(metrics_raw_data, target_quarter)
    
    # Combine all metrics
    combined_metrics = (
        itr_df
        .merge(stockout_df, on=['Product_ID', 'Year_Quarter'])
        .merge(overstock_df, on=['Product_ID', 'Year_Quarter'])
    )
    
    combined_metrics.rename(columns={'stockout_ratio':'Stockout Ratio', 'overstock_cost':'Overstock Cost'}, inplace=True)
    
    return combined_metrics

In [8]:
new_input = metrics_raw_data[metrics_raw_data['Product_ID']=='PROD1001']
refactor_df = preprocess_barpolar(metrics_raw_data)

refactor_df.nsmallest(10, 'ITR')

,Year_Quarter,Product_ID,ITR,Stockout Ratio,Overstock Cost
15,2024-Q4,PROD1015,52.67,16.99,1.132164e+08
122,2024-Q4,PROD1122,55.39,16.28,6.720163e+07
34,2024-Q4,PROD1034,57.83,16.60,1.142092e+08
130,2024-Q4,PROD1130,58.85,16.50,1.106449e+08
144,2024-Q4,PROD1144,59.55,16.66,1.562234e+08
49,2024-Q4,PROD1049,59.73,17.18,8.878109e+07
65,2024-Q4,PROD1065,61.18,16.41,9.862194e+07
40,2024-Q4,PROD1040,61.48,16.56,1.506371e+08
93,2024-Q4,PROD1093,62.07,16.73,1.139501e+08
31,2024-Q4,PROD1031,62.25,16.06,1.348508e+08


In [20]:
from sklearn.preprocessing import MinMaxScaler

ITR_scaler = MinMaxScaler().fit(refactor_df[['ITR']])
refactor_df['ITR_norm'] = ITR_scaler.transform(refactor_df[['ITR']]).round(2)

refactor_df.nlargest(10, 'ITR')
# print(ITR_scaler)

,Year_Quarter,Product_ID,ITR,Stockout Ratio,Overstock Cost,ITR_norm
131,2024-Q4,PROD1131,7.22,17.10,89430471.56,1.00
8,2024-Q4,PROD1008,6.59,16.76,67256919.10,0.79
69,2024-Q4,PROD1069,6.52,16.58,92967948.82,0.77
132,2024-Q4,PROD1132,6.49,16.41,83068787.23,0.76
29,2024-Q4,PROD1029,6.43,16.72,94206295.09,0.74
125,2024-Q4,PROD1125,6.36,16.78,79005301.70,0.71
89,2024-Q4,PROD1089,6.34,17.20,64521354.09,0.71
147,2024-Q4,PROD1147,6.31,16.27,94661486.14,0.70
57,2024-Q4,PROD1057,6.30,16.62,93453966.50,0.69
97,2024-Q4,PROD1097,6.19,16.53,79840908.47,0.66


In [ ]:
def preprocess_barpolar(metrics_raw_data):
    def calculate_inventory_turnover_ratio(metrics_raw_data: pd.DataFrame, target_quarter: str):
        # Filter
        inventory_metrics = metrics_raw_data[metrics_raw_data['Year_Quarter']==target_quarter]

        # Group by quarter and product to get sales and date ranges
        inventory_metrics = inventory_metrics.groupby(
            ['Year_Quarter', 'Product_ID'], 
            as_index=False
        ).agg({
            'Date': ['min', 'max'],
            'Daily_Sales': 'sum'
        })
        
        # Clean up column names
        inventory_metrics.columns = ['_'.join(col).strip() if col[1] else col[0] 
                                for col in inventory_metrics.columns.values]
        
        # Calculate stock levels for start and end dates
        stock_levels = {}
        for date_type in ['min', 'max']:
            stock_levels[date_type] = {
                prod: metrics_raw_data[
                    (metrics_raw_data['Product_ID'] == prod) & 
                    (metrics_raw_data['Date'].isin(inventory_metrics[f'Date_{date_type}']))
                ]['Stock_Level'].sum()
                for prod in inventory_metrics['Product_ID'].unique()
            }
        
        # Add stock levels to dataframe
        inventory_metrics['Earliest_Stock_Sum'] = inventory_metrics['Product_ID'].map(stock_levels['min'])
        inventory_metrics['Latest_Stock_Sum'] = inventory_metrics['Product_ID'].map(stock_levels['max'])
        
        # Calculate ITR
        inventory_metrics['ITR'] = (
            inventory_metrics['Daily_Sales_sum'] / 
            ((inventory_metrics['Latest_Stock_Sum'] + inventory_metrics['Earliest_Stock_Sum']) / 2)
        ).round(2)
        
        return inventory_metrics[['Year_Quarter', 'Product_ID', 'ITR']]

    def calculate_stockout_ratio(metrics_raw_data: pd.DataFrame, target_quarter: str):
        # Filter
        stockout_metrics = metrics_raw_data[stockout_metrics['Year_Quarter'] == target_quarter]

        # Groupby year-quarter and product id
        stockout_metrics = stockout_metrics.groupby(
            ['Year_Quarter', 'Product_ID'], 
            as_index=False
        ).agg({
            'Daily_Sales': 'sum',
            'Lost_Sales': 'sum'
        })
        
        stockout_metrics['stockout_ratio'] = (
            (stockout_metrics['Lost_Sales'] / 
            (stockout_metrics['Lost_Sales'] + stockout_metrics['Daily_Sales'])) * 100
        ).round(2)
        
        return stockout_metrics[['Year_Quarter', 'Product_ID', 'stockout_ratio']]

    def calculate_overstock_cost(metrics_raw_data: pd.DataFrame, target_quarter: str):
        quarterly_data = metrics_raw_data[metrics_raw_data['Year_Quarter'] == target_quarter]
        
        # Calculate daily metrics
        daily_metrics = quarterly_data.groupby(
            ['Year_Quarter', 'Product_ID', 'Date'], 
            as_index=False
        ).agg({
            'Daily_Sales': 'sum',
            'Stock_Level': 'sum',
            'Inventory_Holding_Cost': 'mean'
        })
        
        # Calculate final metrics
        overstock_metrics = (
            daily_metrics.sort_values('Date')
            .groupby(['Year_Quarter', 'Product_ID'], as_index=False)
            .agg({
                'Daily_Sales': 'mean',
                'Stock_Level': 'last',
                'Inventory_Holding_Cost': 'last'
            })
        )
        
        # Calculate overstock cost
        overstock_metrics['overstock_cost'] = (
            (overstock_metrics['Stock_Level'] - overstock_metrics['Daily_Sales']) * 
            overstock_metrics['Inventory_Holding_Cost']
        ).round(2)
        
        return overstock_metrics[['Year_Quarter', 'Product_ID', 'overstock_cost']]

    def normalize_metric(series: pd.Series):
        return (series - series.min()) / (series.max() - series.min())
    
    target_quarter = '2024-Q4'
    itr_df = calculate_inventory_turnover_ratio(metrics_raw_data, target_quarter)
    stockout_df = calculate_stockout_ratio(metrics_raw_data, target_quarter)
    overstock_df = calculate_overstock_cost(metrics_raw_data, target_quarter)
    
    # Combine all metrics
    combined_metrics = (
        itr_df
        .merge(stockout_df, on=['Product_ID', 'Year_Quarter'])
        .merge(overstock_df, on=['Product_ID', 'Year_Quarter'])
    )
    
    # Normalize metrics
    combined_metrics['ITR_norm'] = normalize_metric(combined_metrics['ITR'])
    combined_metrics['stockout_ratio_norm'] = normalize_metric(combined_metrics['stockout_ratio'])
    combined_metrics['overstock_cost_norm'] = normalize_metric(combined_metrics['overstock_cost'])
    combined_metrics.rename(columns={'stockout_ratio':'Stockout Ratio', 'overstock_cost':'Overstock Cost'}, inplace=True)
    
    return combined_metrics

In [15]:
t = preprocess_barpolar(metrics_raw_data)
t.head()

,Year_Quarter,Product_ID,ITR,Stockout Ratio,Overstock Cost,ITR_norm,stockout_ratio_norm,overstock_cost_norm
0,2024-Q4,PROD1000,4.79,16.46,1.157824e+08,0.190000,0.371795,0.455826
1,2024-Q4,PROD1001,5.49,16.51,1.129773e+08,0.423333,0.393162,0.431355
2,2024-Q4,PROD1002,4.81,16.81,1.284583e+08,0.196667,0.521368,0.566412
3,2024-Q4,PROD1003,5.39,17.06,8.744544e+07,0.390000,0.628205,0.208611
4,2024-Q4,PROD1004,6.00,16.37,7.949258e+07,0.593333,0.333333,0.139229


In [20]:
def preprocess_overstock(metrics_raw_data):
    overstock_cost = (
        metrics_raw_data.groupby(['Year_Quarter', 'Product_ID', 'Date'], as_index=False)
        .agg({
            'Daily_Sales': 'sum',
            'Stock_Level': 'sum',
            'Inventory_Holding_Cost': 'mean'
        })
    )
    overstock_cost = (
        overstock_cost.sort_values('Date')
        .groupby(['Year_Quarter'], as_index=False)
        .agg({
            'Date': 'max',
            'Daily_Sales': 'mean',
            'Stock_Level': 'last',
            'Inventory_Holding_Cost': 'last'
        })
    )
    overstock_cost['Inventory_Holding_Cost'] = overstock_cost['Inventory_Holding_Cost'].round(2)
    overstock_cost['Daily_Sales'] = overstock_cost['Daily_Sales'].round(2)
    overstock_cost['Overstock_Cost'] = (overstock_cost['Stock_Level'] - overstock_cost['Daily_Sales']) * overstock_cost['Inventory_Holding_Cost']
    overstock_cost['Overstock_Cost'] = overstock_cost['Overstock_Cost'].round(2)
    overstock_cost.drop(columns=['Daily_Sales', 'Stock_Level', 'Date', 'Inventory_Holding_Cost'], inplace=True)

    return overstock_cost

In [25]:
overstock_cost = preprocess_overstock(metrics_raw_data)
overstock_cost

,Year_Quarter,Overstock_Cost
0,2023-Q1,1.157861e+08
1,2023-Q2,1.216420e+08
2,2023-Q3,1.193282e+08
3,2023-Q4,7.239116e+07
4,2024-Q1,1.034014e+08
5,2024-Q2,9.721783e+07
6,2024-Q3,9.909137e+07
7,2024-Q4,1.364363e+08


In [26]:
overstock_yr_q = sorted(overstock_cost['Year_Quarter'].unique())
latest_overstock_cost = overstock_cost[overstock_cost['Year_Quarter'] == overstock_cost['Year_Quarter'].max()]['Overstock_Cost'].values[0]
latestmin1_overstock_cost = overstock_cost[overstock_cost['Year_Quarter'] == overstock_yr_q[-2]]['Overstock_Cost'].values[0]


In [28]:
latestmin1_overstock_cost

99091374.61